In [927]:
import numpy as np  
import pandas as pd
from pandas import Series, DataFrame
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow.python.ops import confusion_matrix
from tensorflow.python.ops import math_ops

In [928]:
Data=pd.read_csv('C:/Users/dkoch/Documents/tensorflow/R_Input.csv')
Data=Data.fillna(0)

Data['dprio0']=Data['dprio']**0
#Data['dprio2']=Data['dprio']**2
#Data['dprio3']=Data['dprio']**3
Data=Data.rename(columns={'dprio': 'dprio1'})
Data['pax_total']=Data['pax_total'].apply(lambda x: x if x>0 else 0)
Data['pax_total']=Data['pax_total']-Data['group_pax']


direction=pd.get_dummies(Data['direction'],drop_first=True)
direction.columns=['direction:'+x for x in direction.columns]

whichholiday_left_LUX=pd.get_dummies(Data['whichholiday_left_LUX'],drop_first=True)
whichholiday_left_LUX.columns=['whichholiday_left_LUX:'+x for x in whichholiday_left_LUX.columns]

whichholiday_right_LUX=pd.get_dummies(Data['whichholiday_right_LUX'],drop_first=True)
whichholiday_right_LUX.columns=['whichholiday_right_LUX:'+x for x in whichholiday_right_LUX.columns]

Data=pd.concat([Data,direction,whichholiday_left_LUX,whichholiday_right_LUX],axis=1)

Data.index=Data['calendar_date']+'_'+Data['direction']

In [929]:
dprio=Data[['dprio'+str(i) for i in range(0,2)]].drop_duplicates().transpose()
dprio.columns=range(365,-1,-1)
dprio=dprio[[x for x  in range(0,366)]]

X_data=Data.drop_duplicates(subset=['calendar_date','direction'])[[x for x in Data.columns if 'direction:' in x]\
+[x for x in Data.columns if 'whichholiday_left_LUX:' in x]\
+[x for x in Data.columns if 'whichholiday_right_LUX:' in x]\
+['holiday_from_left_LUX','holiday_from_right_LUX','holiday_to_left_LUX','holiday_to_right_LUX']\
+['pax_total']]

Data['concat']=Data['calendar_date']+'_'+Data['direction']
y_data=Data[['concat','dprio1','pax_total']].pivot(index='concat',columns='dprio1',values='pax_total')

In [930]:
X_train=X_data.loc[[x for x in X_data.index if '2019' not in x],:]
X_test=X_data.loc[[x for x in X_data.index if '2019' in x],:]
y_train=y_data.loc[[x for x in y_data.index if '2019' not in x],:]
y_test=y_data.loc[[x for x in y_data.index if '2019' in x],:]

In [931]:
#X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=1)

X_train=X_train.values
X_test=X_test.values
y_train=y_train.values
y_test=y_test.values
dprio.loc['dprio1',:]=dprio.loc['dprio1',:]/365
dprio=dprio.values

In [932]:
n_input=X_train.shape[1]
n_hidden_1=n_input*2
n_hidden_2=2
n_output=y_train.shape[1]

In [933]:
# Parameters
learning_rate = 0.01
training_epochs = 100

X = tf.placeholder("float", [None, n_input])
Z = tf.placeholder("float", [2, 366])
y = tf.placeholder("float", [None, n_output])

keep_prob = tf.placeholder(tf.float32)

# Dictionary of Weights and Biases
weights = {
  'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
  'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
  #'out': tf.Variable(tf.random_normal([n_hidden_2, n_output]))
  #'out': tf.Variable(dprio.reshape(4,366),dtype=tf.float32)  
}

biases = {
  'b1': tf.Variable(tf.random_normal([n_hidden_1])),
  'b2': tf.Variable(tf.random_normal([n_hidden_2])),
  #'out': tf.Variable(tf.random_normal([n_output]))
}

# Model Forwar§d Propagation step
def forward_propagation(x,z):
    # Hidden layer1
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    #layer_2 = tf.nn.relu(layer_2)

    # Output fully connected layer
    #out_layer=tf.add(tf.matmul(layer_2, weights['out']),biases['out'])
    out_layer = tf.matmul(layer_2, z)
    out_layer = tf.math.exp(out_layer)
    return out_layer


# Model Outputs
yhat = forward_propagation(X,Z)

In [934]:
cost = tf.reduce_mean(tf.square(yhat-y))
# our mean squared error cost function
train = tf.train.GradientDescentOptimizer(0.001).minimize(cost)

#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
#cost=tf.losses.mean_squared_error(y, yhat)
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [935]:
c_t=[]
c_test=[]

with tf.Session() as sess:
    # Initiate session and initialize all vaiables
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    #saver.restore(sess,'yahoo_dataset.ckpt')
    for epoch in range(100):
        for i in range(358):
            for j in range(X_train.shape[0]):
                sess.run(train,feed_dict={X:X_train[j].reshape(1,16), y:y_train[j].reshape(1,366), Z:dprio.reshape(2,366)})
            # Run cost and train with each sample
        c_t.append(sess.run(cost, feed_dict={X:X_train,y:y_train,Z:dprio.reshape(2,366)}))
        c_test.append(sess.run(cost, feed_dict={X:X_test,y:y_test,Z:dprio.reshape(2,366)}))
        print('Epoch :',epoch,'Cost :',c_t[epoch])
        pred = sess.run(yhat, feed_dict={X:X_test,Z:dprio.reshape(2,366)})
        print(pred)
#     y_test = denormalize(df_test,y_test)
#     pred = denormalize(df_test,pred)

Epoch : 0 Cost : nan
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(Data.groupby('dprio1')['pax_total'].sum())

In [ ]:
pred[1][0:20]

In [ ]:
pred[10][0:20]

In [ ]:
plt.plot(pred[0][range(365,-1,-1)])

In [ ]:
plt.plot(pred[10][range(365,-1,-1)])

In [ ]:
plt.plot(pred[40][range(365,-1,-1)])

In [ ]:
np.cumsum(pred[0][range(365,-1,-1)])

In [ ]:
np.cumsum(pred[10][range(365,-1,-1)])

In [ ]:
pred